<a href="https://colab.research.google.com/github/jkunalv6/Machine-Learning/blob/main/Aluminium_predictv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [423]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
!pip install torchmetrics
from torchmetrics import Accuracy



In [424]:
import cuda

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [425]:
data = pd.read_csv('/content/al_data.csv')
data.head()

,Unnamed: 0,Processing,Ag,Al,B,Be,Bi,Cd,Co,Cr,...,Si,Sn,Ti,V,Zn,Zr,Elongation (%),Tensile Strength (MPa),Yield Strength (MPa),class
0,0,Solutionised + Artificially peak aged,0.0,0.88011,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00034,0.0,0.0,0.0,0.0768,0.0012,16.8,651.6,583.3,2
1,1,Solutionised + Artificially over aged,0.0,0.88011,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00034,0.0,0.0,0.0,0.0768,0.0012,15.4,557.0,513.0,4
2,2,Solutionised + Artificially peak aged,0.0,0.99450,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0000,0.0000,10.5,320.0,300.0,2
3,3,Solutionised + Artificially peak aged,0.0,0.99250,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0000,0.0000,4.5,280.0,265.0,2
4,4,Solutionised + Artificially peak aged,0.0,0.99000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0000,0.0000,7.0,325.0,290.0,2


In [426]:
#dropping = data[data['Processing'] != 'No Processing'].index
#data.drop(dropping, inplace = True)
#data

In [427]:
data.drop(['class'], axis=1)

,Unnamed: 0,Processing,Ag,Al,B,Be,Bi,Cd,Co,Cr,...,Sc,Si,Sn,Ti,V,Zn,Zr,Elongation (%),Tensile Strength (MPa),Yield Strength (MPa)
0,0,Solutionised + Artificially peak aged,0.0,0.880110,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0000,0.00034,0.0,0.000000,0.0,0.076800,0.0012,16.8,651.6,583.3
1,1,Solutionised + Artificially over aged,0.0,0.880110,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0000,0.00034,0.0,0.000000,0.0,0.076800,0.0012,15.4,557.0,513.0
2,2,Solutionised + Artificially peak aged,0.0,0.994500,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0055,0.00000,0.0,0.000000,0.0,0.000000,0.0000,10.5,320.0,300.0
3,3,Solutionised + Artificially peak aged,0.0,0.992500,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0075,0.00000,0.0,0.000000,0.0,0.000000,0.0000,4.5,280.0,265.0
4,4,Solutionised + Artificially peak aged,0.0,0.990000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0100,0.00000,0.0,0.000000,0.0,0.000000,0.0000,7.0,325.0,290.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,1149,No Processing,0.0,0.992713,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0000,0.00004,0.0,0.000016,0.0,0.000006,0.0000,32.0,110.0,105.0
1150,1150,No Processing,0.0,0.967475,0.0,0.0,0.0,0.0,0.0,0.000020,...,0.0000,0.00087,0.0,0.000300,0.0,0.000025,0.0000,14.2,195.0,58.0
1151,1151,No Processing,0.0,0.966676,0.0,0.0,0.0,0.0,0.0,0.000014,...,0.0000,0.00110,0.0,0.000230,0.0,0.000010,0.0014,10.7,210.0,110.0
1152,1152,Strain Harderned (Hard),0.0,0.955000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0050,0.00000,0.0,0.000000,0.0,0.000000,0.0000,NaN,460.0,414.0


In [428]:
data.isnull().sum()

Unnamed: 0                  0
Processing                  0
Ag                          0
Al                          0
B                           0
Be                          0
Bi                          0
Cd                          0
Co                          0
Cr                          0
Cu                          0
Er                          0
Eu                          0
Fe                          0
Ga                          0
Li                          0
Mg                          0
Mn                          0
Ni                          0
Pb                          0
Sc                          0
Si                          0
Sn                          0
Ti                          0
V                           0
Zn                          0
Zr                          0
Elongation (%)            108
Tensile Strength (MPa)     33
Yield Strength (MPa)      109
class                       0
dtype: int64

In [429]:
data= data.dropna()
data = data.drop(['class', 'Unnamed: 0'], axis = 1)

In [430]:
X = data.copy()
X = X.drop(['Elongation (%)','Tensile Strength (MPa)','Yield Strength (MPa)'], axis =1)

In [431]:
x_dummy = pd.get_dummies(X['Processing'], drop_first = True)
x_dummy = x_dummy.astype(int)
X = pd.concat([X, x_dummy], axis = 1)
X.drop(['Processing'], axis = 1, inplace = True)
X.head()


,Ag,Al,B,Be,Bi,Cd,Co,Cr,Cu,Er,...,Zr,Naturally aged,No Processing,Solutionised,Solutionised + Artificially peak aged,Solutionised + Artificially over aged,Solutionised + Cold Worked + Naturally aged,Solutionised + Naturally aged,Strain Harderned (Hard),Strain hardened
0,0.0,0.88011,0.0,0.0,0.0,0.0,0.0,0.0,0.0198,0.0,...,0.0012,0,0,0,1,0,0,0,0,0
1,0.0,0.88011,0.0,0.0,0.0,0.0,0.0,0.0,0.0198,0.0,...,0.0012,0,0,0,0,1,0,0,0,0
2,0.0,0.99450,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0,0,0,1,0,0,0,0,0
3,0.0,0.99250,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0,0,0,1,0,0,0,0,0
4,0.0,0.99000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0,0,0,1,0,0,0,0,0


In [432]:
y = data[[ 'Tensile Strength (MPa)']]
y.head()

,Tensile Strength (MPa)
0,651.6
1,557.0
2,320.0
3,280.0
4,325.0


In [433]:
X_train , X_test , y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)
X_train

,Ag,Al,B,Be,Bi,Cd,Co,Cr,Cu,Er,...,Zr,Naturally aged,No Processing,Solutionised,Solutionised + Artificially peak aged,Solutionised + Artificially over aged,Solutionised + Cold Worked + Naturally aged,Solutionised + Naturally aged,Strain Harderned (Hard),Strain hardened
1068,0.0,0.942800,0.0,0.000000,0.0,0.0,0.0,0.000000,0.019000,0.0,...,0.001200,0,0,0,0,1,0,0,0,0
113,0.0,0.989000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0,0,0,0,0,0,0,1,0
346,0.0,0.983000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0,1,0,0,0,0,0,0,0
1131,0.0,0.927474,0.0,0.000000,0.0,0.0,0.0,0.000000,0.063066,0.0,...,0.001752,0,0,0,0,1,0,0,0,0
510,0.0,0.922500,0.0,0.000000,0.0,0.0,0.0,0.001000,0.044000,0.0,...,0.000000,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,0.0,0.969485,0.0,0.000000,0.0,0.0,0.0,0.000000,0.001251,0.0,...,0.000000,0,0,0,0,0,0,0,1,0
359,0.0,0.972500,0.0,0.000000,0.0,0.0,0.0,0.002500,0.000000,0.0,...,0.000000,0,1,0,0,0,0,0,0,0
1066,0.0,0.944849,0.0,0.000001,0.0,0.0,0.0,0.000000,0.024500,0.0,...,0.001700,0,0,0,0,1,0,0,0,0
548,0.0,0.973022,0.0,0.000000,0.0,0.0,0.0,0.001952,0.002753,0.0,...,0.000000,0,0,0,0,0,0,1,0,0


First make a model with 4 layers : input , hidden , hidden and output layer.


MOdel 1 params
Input features = X columns
out features = y columns
Hidden layer1 = 2 = 21
activation functions
ReLU
ReLU
ReLU


In [434]:
torch.manual_seed(42)

class Model1(nn.Module):
  def __init__ (self, input_features = 34, output_features=1, hidden_units=21):
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features= input_features, out_features = hidden_units),
        nn.ReLU(),
        #nn.Dropout(p=0.5),
        nn.Linear(in_features=hidden_units, out_features = hidden_units),
        nn.ReLU(),
        #nn.Dropout(p=0.5),
        nn.Linear(in_features=hidden_units, out_features = hidden_units),
        nn.ReLU(),
        #nn.Dropout(p=0.5),
        nn.Linear(in_features=hidden_units, out_features = output_features)
    )

  def forward(self, x):
          return self.linear_layer_stack(x)


model1 = Model1().to(device)

loss_fn = nn.L1Loss().to(device)
optimizer = torch.optim.Adam(params = model1.parameters(), lr = 0.1, weight_decay= 0.1)
from torchmetrics import MeanSquaredError
mse = MeanSquaredError().to(device)


In [435]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [436]:
"""
X_train = torch.from_numpy(X_train).type(torch.float).to(device)
y_train = torch.from_numpy(y_train).type(torch.float).to(device)
X_test = torch.from_numpy(X_test).type(torch.float).to(device)
y_test = torch.from_numpy(y_test).type(torch.float).to(device)
"""

'\nX_train = torch.from_numpy(X_train).type(torch.float).to(device)\ny_train = torch.from_numpy(y_train).type(torch.float).to(device)\nX_test = torch.from_numpy(X_test).type(torch.float).to(device)\ny_test = torch.from_numpy(y_test).type(torch.float).to(device)\n'

In [437]:
"""
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 100000


for epoch in range(epochs):
  model1.train()
  y_pred = model1(X_train)
  loss = loss_fn(y_pred, y_train)
  error = mse(y_pred.reshape(-1), y_train.reshape(-1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model1.eval()
  with torch.inference_mode():
    test_pred = model1(X_test)
    test_loss = loss_fn(test_pred, y_test)
    test_mse = mse(test_pred.reshape(-1), y_test.reshape(-1))
  if epoch % 10000 ==0 :
    print(f"Epoch: {epoch} | Loss: {loss:.4f}, Error: {error:.2f}% | Test Loss: {test_loss:.4f}, Test error: {test_mse:.2f}%")

    """

'\ntorch.manual_seed(42)\ntorch.cuda.manual_seed(42)\n\nepochs = 100000\n\n\nfor epoch in range(epochs):\n  model1.train()\n  y_pred = model1(X_train)\n  loss = loss_fn(y_pred, y_train)\n  error = mse(y_pred.reshape(-1), y_train.reshape(-1))\n  optimizer.zero_grad()\n  loss.backward()\n  optimizer.step()\n\n  model1.eval()\n  with torch.inference_mode():\n    test_pred = model1(X_test)\n    test_loss = loss_fn(test_pred, y_test)\n    test_mse = mse(test_pred.reshape(-1), y_test.reshape(-1))\n  if epoch % 10000 ==0 :\n    print(f"Epoch: {epoch} | Loss: {loss:.4f}, Error: {error:.2f}% | Test Loss: {test_loss:.4f}, Test error: {test_mse:.2f}%")\n\n    '

Tried using multiple linear regression and the results came out pretty great


In [438]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
import math

In [439]:
model2 = LinearRegression()
model2.fit(X_train, y_train)

LinearRegression()

In [440]:

predictions = model2.predict(X_test)
print( 'mean_squared_error : ', math.sqrt(mean_squared_error(y_test, predictions)))
print( 'mean_absolute_error : ', mean_absolute_error(y_test, predictions))


mean_squared_error :  51.881186941232784
mean_absolute_error :  39.951068572073375


In [441]:

first_test_instance = X_test[0]
predicted_value = model2.predict([first_test_instance])
print("Predicted value:", predicted_value)
print("Actual Values" , y_test[0])

Predicted value: [[364.62221122]]
Actual Values [230.]


In [442]:
predictions_df = pd.DataFrame(predictions, columns=[ 'Tensile Strength (MPa)'])
predictions_df

,Tensile Strength (MPa)
0,364.621766
1,431.762349
2,201.800435
3,322.852256
4,378.911557
...,...
89,469.927854
90,212.221864
91,201.800435
92,466.227528


In [443]:
y_test_df = pd.DataFrame(y_test, columns=[ 'Tensile Strength (MPa)'])
y_test_df

,Tensile Strength (MPa)
0,230.000000
1,400.000000
2,220.632224
3,314.000000
4,420.000000
...,...
89,531.000000
90,184.000000
91,220.000000
92,642.000000


In [444]:
from sklearn.svm import SVR
model3 = SVR(kernel="poly" , degree = 5)
model3.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(degree=5, kernel='poly')

In [445]:
preds3 = model3.predict(X_test)
print( 'mean_squared_error : ', math.sqrt(mean_squared_error(y_test, preds3)))
print( 'mean_absolute_error : ', mean_absolute_error(y_test, preds3))

mean_squared_error :  169.25293547386806
mean_absolute_error :  133.73051672036314


In [446]:
X_train = torch.from_numpy(X_train).type(torch.float).to(device)
y_train = torch.from_numpy(y_train).type(torch.float).to(device)
X_test = torch.from_numpy(X_test).type(torch.float).to(device)
y_test = torch.from_numpy(y_test).type(torch.float).to(device)

ran 100000 epochs , the best result is at 6000
Need to look into
1. Dropout layers
2. less weight decay
3. more layers ?
4. More neurons on the inner layers
5. Leaky relu ?

In [447]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 6000


for epoch in range(epochs):
  model1.train()
  y_pred = model1(X_train)
  loss = loss_fn(y_pred, y_train)
  error = mse(y_pred.reshape(-1), y_train.reshape(-1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model1.eval()
  with torch.inference_mode():
    test_pred = model1(X_test)
    test_loss = loss_fn(test_pred, y_test)
    test_mse = mse(test_pred.reshape(-1), y_test.reshape(-1))
  if epoch % 1000 ==0 :
    print(f"Epoch: {epoch} | Loss: {loss:.4f}, Error: {math.sqrt(error):.2f} | Test Loss: {test_loss:.4f}, Test error: {math.sqrt(test_mse):.2f}")

Epoch: 0 | Loss: 351.1151, Error: 382.05 | Test Loss: 337.0752, Test error: 374.42
Epoch: 1000 | Loss: 26.8097, Error: 38.57 | Test Loss: 32.6899, Test error: 42.80
Epoch: 2000 | Loss: 23.3891, Error: 35.67 | Test Loss: 26.2373, Test error: 36.75
Epoch: 3000 | Loss: 22.3072, Error: 34.49 | Test Loss: 28.4251, Test error: 39.46
Epoch: 4000 | Loss: 21.1666, Error: 32.59 | Test Loss: 26.6297, Test error: 38.91
Epoch: 5000 | Loss: 22.3103, Error: 33.85 | Test Loss: 23.5362, Test error: 33.44


In [448]:
PATH = 'AL_MODEL_NN1'
torch.save(model1.state_dict(), PATH)

In [449]:
torch.manual_seed(42)

class Model2(nn.Module):
  def __init__ (self, input_features = 34, output_features=1, hidden_units=21):
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features= input_features, out_features = hidden_units),
        nn.ELU(),
        nn.Linear(in_features=hidden_units, out_features = hidden_units),
        nn.ELU(),
        nn.Linear(in_features=hidden_units, out_features = hidden_units),
        nn.ELU(),
        nn.Linear(in_features=hidden_units, out_features = output_features)
    )

  def forward(self, x):
          return self.linear_layer_stack(x)


model2 = Model2().to(device)

optimizer2 = torch.optim.Adam(params = model2.parameters(), lr = 0.1, weight_decay= 0.1)

In [450]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 60000


for epoch in range(epochs):
  model2.train()
  y_pred = model2(X_train)
  loss = loss_fn(y_pred, y_train)
  error = mse(y_pred.reshape(-1), y_train.reshape(-1))
  optimizer2.zero_grad()
  loss.backward()
  optimizer2.step()

  model2.eval()
  with torch.inference_mode():
    test_pred = model2(X_test)
    test_loss = loss_fn(test_pred, y_test)
    test_mse = mse(test_pred.reshape(-1), y_test.reshape(-1))
  if epoch % 1000 ==0 :
    print(f"Epoch: {epoch} | Loss: {loss:.4f}, Error: {math.sqrt(error):.2f} | Test Loss: {test_loss:.4f}, Test error: {math.sqrt(test_mse):.2f}")

Epoch: 0 | Loss: 351.1469, Error: 382.08 | Test Loss: 336.8187, Test error: 374.20
Epoch: 1000 | Loss: 22.9956, Error: 32.95 | Test Loss: 28.0913, Test error: 40.13
Epoch: 2000 | Loss: 22.2442, Error: 33.08 | Test Loss: 24.4539, Test error: 34.81
Epoch: 3000 | Loss: 21.5818, Error: 31.92 | Test Loss: 25.0493, Test error: 35.55
Epoch: 4000 | Loss: 20.3103, Error: 31.28 | Test Loss: 25.6038, Test error: 35.77
Epoch: 5000 | Loss: 21.1840, Error: 31.61 | Test Loss: 27.3147, Test error: 37.44
Epoch: 6000 | Loss: 19.8314, Error: 30.38 | Test Loss: 25.6014, Test error: 34.88
Epoch: 7000 | Loss: 20.7849, Error: 31.03 | Test Loss: 25.7308, Test error: 36.98
Epoch: 8000 | Loss: 20.6540, Error: 30.84 | Test Loss: 26.4312, Test error: 37.41
Epoch: 9000 | Loss: 19.8939, Error: 30.17 | Test Loss: 25.2524, Test error: 35.57
Epoch: 10000 | Loss: 19.9296, Error: 30.67 | Test Loss: 25.1857, Test error: 34.92
Epoch: 11000 | Loss: 19.9838, Error: 30.49 | Test Loss: 23.8434, Test error: 34.57
Epoch: 12000 

Tried changing dropout layers, leakyrelu, changing weight decay(both increase and decrease) and learning rate
nothing was able to get as close to the current best rms error of 33.44

ELU activation layers with epoch 34000 decreased the error to 32.17
